In [3]:
import deepmatcher as dm
hybrid = dm.MatchingModel(attr_summarizer='hybrid')
hybrid.load_state('models/hybrid_model.pth')

In [4]:
import pandas as pd
train_df = pd.read_csv('sample_data/itunes-amazon/train.csv')
validation_df = pd.read_csv('sample_data/itunes-amazon/validation.csv')
test_df = pd.read_csv('sample_data/itunes-amazon/test.csv')

In [5]:
columns = list(train_df)

In [6]:
selected_columns = columns[2:]
selected_columns

['left_Song_Name',
 'left_Artist_Name',
 'left_Album_Name',
 'left_Genre',
 'left_Price',
 'left_CopyRight',
 'left_Time',
 'left_Released',
 'right_Song_Name',
 'right_Artist_Name',
 'right_Album_Name',
 'right_Genre',
 'right_Price',
 'right_CopyRight',
 'right_Time',
 'right_Released']

In [10]:
import random as rd
def update_dictionary (dictionary,start_id,end_id,df,columns):
    for idx,row in df.iterrows():
        for col in columns:
            current_word = row[col]
            if not(isinstance(current_word,str)):
                current_word = str(current_word)
            for word in current_word.split(" "):
                if word not in dictionary.keys():
                    dictionary[word] = rd.randint(start_id,end_id)

In [11]:
import random as rd
def update_dictionary2 (dictionary,start_id,df,columns):
    curr_id = start_id
    for idx,row in df.iterrows():
        for col in columns:
            current_word = row[col]
            if not(isinstance(current_word,str)):
                current_word = str(current_word)
            for word in current_word.split(" "):
                if word not in dictionary.keys():
                    dictionary[word] = curr_id
                    curr_id += 1

In [12]:
dataset_df = pd.concat([train_df,validation_df,test_df])
dataset_df.shape

(539, 18)

In [15]:
wordDictionary = {}

In [14]:
## run this cell if you prefer random encoding
update_dictionary(wordDictionary,0,2200,dataset_df,selected_columns)                

In [17]:
update_dictionary2(wordDictionary,0,dataset_df,selected_columns)

In [18]:
def convert_attribute_to_idlist(att,dictionary=wordDictionary):
    if not(isinstance(att,str)):
        return att
    new_attribute =""
    tokens = att.split(" ")
    for token in tokens:
        tokenid = dictionary[token]
        new_attribute = new_attribute + str(tokenid)+ " "
    return new_attribute

In [25]:
train_df_numericencoding = train_df.applymap(lambda c: convert_attribute_to_idlist(c))

In [26]:
validation_df_numericencoding = validation_df.applymap(lambda c: convert_attribute_to_idlist(c))
test_df_numericencoding = test_df.applymap(lambda c:convert_attribute_to_idlist(c))

In [27]:
train_df_numericencoding.to_csv('sample_data/itunes-amazon/exp4/train_numericencoding.csv',index=False)
validation_df_numericencoding.to_csv('sample_data/itunes-amazon/exp4/validation_numericencoding.csv',index=False)
test_df_numericencoding.to_csv('sample_data/itunes-amazon/exp4/test_numericencoding.csv',index=False)

In [28]:
train_pos = train_df_numericencoding[train_df_numericencoding['label']==1]
val_pos = validation_df_numericencoding[validation_df_numericencoding['label']==1]
test_pos = test_df_numericencoding[test_df_numericencoding['label']==1]

In [29]:
train_neg = train_df_numericencoding[train_df_numericencoding['label']==0]
val_neg = validation_df_numericencoding[validation_df_numericencoding['label']==0]
test_neg = test_df_numericencoding[test_df_numericencoding['label']==0]

In [30]:
all_pos = pd.concat([train_pos,val_pos,test_pos])
all_neg = pd.concat([train_neg,val_neg,test_neg])

In [23]:
all_pos.head()

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
1,287,1,371,613 896,2074,2005 2136 432 2136 607 1433 333 1433 664 2136 ...,1413 1147,1867 613 896 1765 1271 274 1250 1619 736 1000 ...,122,430,371,613 896,2074 1660 1659 2111,2005,1413 1147,1623 1104 539 444 262 478,122,793 1437 2136 1867
3,181,1,495 926 292 1623 1212 2177 539 539,1277,495 926 292 1623 1682 539 1401 1583,2061 2136 432 2136 670 2136 901,1413 586,1472 668 392 702,250,1181,495 926 292 1623 0 1524 539 539,1277,495 926 292 1660 0 1524 539 2111,2061 798 670,1413 586,1472 668 392 702,1148,1798 331 2136 1006
7,493,1,777 1224 1250 1991 2125 1968,83 1144 1420,777 1224 1250 1991 2125 1968,1170 2136 432 2136 2094 1149 2136 491 492 2136...,1413 586,720 2117 1472 369 399 652 431 2136 860,2082,2076,777 1224 1893 1009 2125 1968,83 1144 1420,777 1224 1893 1009 2125 1968,1170,1413 586,1623 1453 539 1472 369 399 652 431 2136 860,1536,1647 1755 2136 1472
13,216,1,341 718 1674 1777,297 1500,581 1991 2111,1170 2136 432 2136 2094 1149 2136 2021 1170,1413 586,1006 1158 699 478,933,646,341 718 1674 1777,297 1500,581 1991 2111,1170,1413 586,1623 1453 539 1006 1158 699 478 2136 1765 1271...,933,1677 1793 2136 1006
15,53,1,1925 954 487 128 1968 1990 787 1404,780 626,128 663 1250 1991 1389 1401 1583,1170 2136 432,1413 586,1006 25 432 1774,1278,875,1925 954 487 128 1968 1990 787 1404,780 626,128 663 1250 1991 1389 1623 1583 539,1170,1413 586,1623 1453 539 1472 1835 1739 478 702,1278,1913 1996 2136 1006


In [31]:
all_pos.to_csv('sample_data/itunes-amazon/exp4/all_positives_numericencoding.csv',index=False)
all_neg.to_csv('sample_data/itunes-amazon/exp4/all_negatives_numericencoding.csv',index=False)